# 1. IMPORTACIÓN DE LIBRERÍAS

Importamos las librerias necesarias para comenzar a trabajar con los datos

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [13]:
import joblib
from sklearn import tree
from sklearn.tree import export_text
from xgboost import XGBClassifier
from subprocess import check_call
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score, confusion_matrix
from sklearn import tree
from sklearn.tree import export_text
from xgboost import XGBClassifier
from subprocess import check_call
from IPython.display import Image as PImage

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras import metrics

///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# REGRESION LOGISTICA

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (roc_curve, roc_auc_score, precision_score, recall_score,
                           f1_score, accuracy_score, confusion_matrix)
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import json

# Cargar datos
df_mora_prestamos = pd.read_csv('D:/Tesis/Definitivos/nuevo/df_mora_prestamospersonales.csv')
X = df_mora_prestamos.drop('Morosidad', axis=1)
y = df_mora_prestamos['Morosidad']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Escalar datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicar SMOTE a los datos de entrenamiento
print("Aplicando SMOTE a los datos de entrenamiento...")
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

# Configurar y entrenar modelo
model = LogisticRegression(
    C=10,
    penalty='l1',
    class_weight='balanced',
    max_iter=100,
    fit_intercept=True,
    solver='liblinear',
    random_state=42
)

print("Entrenando modelo de Regresión Logística...")
model.fit(X_train_balanced, y_train_balanced)

# Realizar predicciones
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Calcular métricas
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

metrics = {
    'ROC_AUC': roc_auc_score(y_test, y_pred_proba),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'Specificity': specificity,
    'F1_Score': f1_score(y_test, y_pred),
    'Accuracy': accuracy_score(y_test, y_pred)
}

# Calcular datos para curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Crear directorio para resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_dir = f'D:/Tesis/Definitivos/nuevo/logistic_regression_smote_results_{timestamp}'
os.makedirs(result_dir, exist_ok=True)

# Guardar datos ROC para comparación posterior
roc_data = {
    'model': 'Logistic_Regression',
    'fpr': fpr.tolist(),
    'tpr': tpr.tolist(),
    'roc_auc': roc_auc,
    'thresholds': thresholds.tolist()
}

with open(f'{result_dir}/roc_data.json', 'w') as f:
    json.dump(roc_data, f)

# 1. Graficar y guardar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Regresión Logística con SMOTE')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.savefig(f'{result_dir}/confusion_matrix.png')
plt.close()

# 2. Graficar y guardar curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Regresión Logística con SMOTE')
plt.legend(loc="lower right")
plt.savefig(f'{result_dir}/roc_curve.png')
plt.close()

# 3. Graficar Recall y Especificidad
plt.figure(figsize=(8, 6))
metrics_plot = [metrics['Recall'], metrics['Specificity']]
plt.bar(['Recall (Sensibilidad)', 'Especificidad'], metrics_plot, color=['skyblue', 'lightgreen'])
plt.title('Recall vs Especificidad - Regresión Logística con SMOTE')
plt.ylim([0, 1])
for i, v in enumerate(metrics_plot):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.savefig(f'{result_dir}/recall_specificity.png')
plt.close()

# Guardar todas las métricas y resultados
results = {
    'metrics': metrics,
    'confusion_matrix': cm.tolist(),
    'class_distribution': {
        'original_train': {
            'class_0': int(sum(y_train == 0)),
            'class_1': int(sum(y_train == 1))
        },
        'balanced_train': {
            'class_0': int(sum(y_train_balanced == 0)),
            'class_1': int(sum(y_train_balanced == 1))
        }
    }
}

with open(f'{result_dir}/all_results.json', 'w') as f:
    json.dump(results, f, indent=4)

# Imprimir resultados en consola
print("\nResultados del modelo de Regresión Logística con SMOTE:")
print("="*40)
print(f"\nDistribución original de clases en entrenamiento:")
print(f"Clase 0: {sum(y_train == 0)}")
print(f"Clase 1: {sum(y_train == 1)}")
print(f"\nDistribución después de SMOTE:")
print(f"Clase 0: {sum(y_train_balanced == 0)}")
print(f"Clase 1: {sum(y_train_balanced == 1)}")

print("\nMétricas principales:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nMatriz de Confusión:")
print(cm)

print(f"\nResultados guardados en: {result_dir}")

Aplicando SMOTE a los datos de entrenamiento...
Entrenando modelo de Regresión Logística...

Resultados del modelo de Regresión Logística con SMOTE:

Distribución original de clases en entrenamiento:
Clase 0: 1072714
Clase 1: 65670

Distribución después de SMOTE:
Clase 0: 1072714
Clase 1: 1072714

Métricas principales:
ROC_AUC: 0.9949
Precision: 0.6962
Recall: 0.9683
Specificity: 0.9741
F1_Score: 0.8100
Accuracy: 0.9738

Matriz de Confusión:
[[447841  11894]
 [   891  27254]]

Resultados guardados en: D:/Tesis/Definitivos/nuevo/logistic_regression_smote_results_20250203_234815


# RANDOM FOREST

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_curve, auc, roc_auc_score, precision_score, 
                           recall_score, f1_score, accuracy_score, confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import json

# Cargar datos
df_mora_prestamos = pd.read_csv('D:/Tesis/Definitivos/nuevo/df_mora_prestamospersonales.csv')
X = df_mora_prestamos.drop('Morosidad', axis=1)
y = df_mora_prestamos['Morosidad']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Configurar y entrenar modelo con los mejores hiperparámetros
model = RandomForestClassifier(
    n_estimators=900,
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    bootstrap=True,
    random_state=42
)

print("Entrenando modelo Random Forest...")
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calcular métricas
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

metrics = {
    'ROC_AUC': roc_auc_score(y_test, y_pred_proba),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'Specificity': specificity,
    'F1_Score': f1_score(y_test, y_pred),
    'Accuracy': accuracy_score(y_test, y_pred)
}

# Calcular datos para curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Crear directorio para resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_dir = f'D:/Tesis/Definitivos/nuevo/random_forest_results_{timestamp}'
os.makedirs(result_dir, exist_ok=True)

# 1. Graficar y guardar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Random Forest')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.savefig(f'{result_dir}/confusion_matrix.png')
plt.close()

# 2. Graficar y guardar curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Random Forest')
plt.legend(loc="lower right")
plt.savefig(f'{result_dir}/roc_curve.png')
plt.close()

# 3. Graficar Recall y Especificidad
plt.figure(figsize=(8, 6))
metrics_plot = [metrics['Recall'], metrics['Specificity']]
plt.bar(['Recall (Sensibilidad)', 'Especificidad'], metrics_plot, color=['skyblue', 'lightgreen'])
plt.title('Recall vs Especificidad - Random Forest')
plt.ylim([0, 1])
for i, v in enumerate(metrics_plot):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.savefig(f'{result_dir}/recall_specificity.png')
plt.close()

# Guardar todos los resultados
results = {
    'model_name': 'Random_Forest',
    'hyperparameters': {
        'n_estimators': 900,
        'max_depth': 20,
        'min_samples_split': 5,
        'min_samples_leaf': 1,
        'max_features': 'log2',
        'class_weight': 'balanced_subsample',
        'bootstrap': True
    },
    'metrics': metrics,
    'confusion_matrix': cm.tolist(),
    'class_distribution': {
        'train': {
            'class_0': int(sum(y_train == 0)),
            'class_1': int(sum(y_train == 1))
        }
    },
    'roc_data': {
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist(),
        'roc_auc': roc_auc,
        'thresholds': thresholds.tolist()
    },
    'sensitivity_specificity': {
        'recall': float(metrics['Recall']),
        'specificity': float(metrics['Specificity']),
        'confusion_matrix_values': {
            'tn': int(tn),
            'fp': int(fp),
            'fn': int(fn),
            'tp': int(tp)
        }
    }
}

# Guardar resultados en JSON
with open(f'{result_dir}/model_results.json', 'w') as f:
    json.dump(results, f, indent=4)

# Imprimir resultados en consola
print("\nResultados del modelo Random Forest:")
print("="*40)
print("\nMétricas principales:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nMatriz de Confusión:")
print(cm)

print(f"\nResultados guardados en: {result_dir}")

Entrenando modelo Random Forest...

Resultados del modelo Random Forest:

Métricas principales:
ROC_AUC: 0.9996
Precision: 0.9801
Recall: 0.9669
Specificity: 0.9988
F1_Score: 0.9735
Accuracy: 0.9970

Matriz de Confusión:
[[459183    552]
 [   931  27214]]

Resultados guardados en: D:/Tesis/Definitivos/nuevo/random_forest_results_20250204_005324


# REDES NEURONALES

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
from sklearn.metrics import (roc_curve, auc, roc_auc_score, precision_score, 
                          recall_score, f1_score, accuracy_score, confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import json

# Cargar datos
df_mora_prestamos = pd.read_csv('D:/Tesis/Definitivos/nuevo/df_mora_prestamospersonales.csv')
X = df_mora_prestamos.drop('Morosidad', axis=1)
y = df_mora_prestamos['Morosidad']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.3, stratify=y, random_state=42
)

# Escalar datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Configurar y entrenar modelo con los mejores hiperparámetros
def create_model():
   model = Sequential([
       Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
       Dropout(0.1),
       Dense(64, activation='relu'),
       Dropout(0.1),
       Dense(16, activation='relu'),
       Dropout(0.1),
       Dense(1, activation='sigmoid')
   ])
   model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'])
   return model

print("Entrenando modelo de Red Neuronal...")
model = create_model()
history = model.fit(X_train_scaled, y_train,
                  batch_size=64,
                  epochs=100,
                  verbose=0)

# Realizar predicciones
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calcular métricas
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

metrics = {
   'ROC_AUC': roc_auc_score(y_test, y_pred_proba),
   'Precision': precision_score(y_test, y_pred),
   'Recall': recall_score(y_test, y_pred),
   'Specificity': specificity,
   'F1_Score': f1_score(y_test, y_pred),
   'Accuracy': accuracy_score(y_test, y_pred)
}

# Calcular datos para curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Crear directorio para resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_dir = f'D:/Tesis/Definitivos/nuevo/neural_network_results_{timestamp}'
os.makedirs(result_dir, exist_ok=True)

# 1. Graficar y guardar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Red Neuronal')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.savefig(f'{result_dir}/confusion_matrix.png')
plt.close()

# 2. Graficar y guardar curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Red Neuronal')
plt.legend(loc="lower right")
plt.savefig(f'{result_dir}/roc_curve.png')
plt.close()

# 3. Graficar Recall y Especificidad
plt.figure(figsize=(8, 6))
metrics_plot = [metrics['Recall'], metrics['Specificity']]
plt.bar(['Recall (Sensibilidad)', 'Especificidad'], metrics_plot, color=['skyblue', 'lightgreen'])
plt.title('Recall vs Especificidad - Red Neuronal')
plt.ylim([0, 1])
for i, v in enumerate(metrics_plot):
   plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.savefig(f'{result_dir}/recall_specificity.png')
plt.close()

# Guardar todos los resultados
results = {
   'model_name': 'Neural_Network',
   'hyperparameters': {
       'architecture': [128, 64, 16],
       'dropout_rate': 0.1,
       'batch_size': 64,
       'learning_rate': 0.001,
       'epochs': 100,
       'optimizer': 'adam',
       'loss': 'binary_crossentropy'
   },
   'metrics': metrics,
   'confusion_matrix': cm.tolist(),
   'class_distribution': {
       'train': {
           'class_0': int(sum(y_train == 0)),
           'class_1': int(sum(y_train == 1))
       }
   },
   'roc_data': {
       'fpr': fpr.tolist(),
       'tpr': tpr.tolist(),
       'roc_auc': roc_auc,
       'thresholds': thresholds.tolist()
   },
   'sensitivity_specificity': {
       'recall': float(metrics['Recall']),
       'specificity': float(metrics['Specificity']),
       'confusion_matrix_values': {
           'tn': int(tn),
           'fp': int(fp),
           'fn': int(fn),
           'tp': int(tp)
       }
   }
}

# Guardar resultados en JSON
with open(f'{result_dir}/model_results.json', 'w') as f:
   json.dump(results, f, indent=4)

# Imprimir resultados en consola
print("\nResultados del modelo Red Neuronal:")
print("="*40)
print("\nMétricas principales:")
for metric, value in metrics.items():
   print(f"{metric}: {value:.4f}")

print("\nMatriz de Confusión:")
print(cm)

print(f"\nResultados guardados en: {result_dir}")

Entrenando modelo de Red Neuronal...


C:\Users\Usuario\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15247/15247 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step

Resultados del modelo Red Neuronal:

Métricas principales:
ROC_AUC: 0.9994
Precision: 0.9904
Recall: 0.9748
Specificity: 0.9994
F1_Score: 0.9825
Accuracy: 0.9980

Matriz de Confusión:
[[459468    267]
 [   709  27436]]

Resultados guardados en: D:/Tesis/Definitivos/nuevo/neural_network_results_20250204_020451


# XGBOOST

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import (roc_curve, auc, roc_auc_score, precision_score, 
                           recall_score, f1_score, accuracy_score, confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import json

# Cargar datos
df_mora_prestamos = pd.read_csv('D:/Tesis/Definitivos/nuevo/df_mora_prestamospersonales.csv')
X = df_mora_prestamos.drop('Morosidad', axis=1)
y = df_mora_prestamos['Morosidad']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Configurar y entrenar modelo con los mejores hiperparámetros
model = xgb.XGBClassifier(
    subsample=0.8,
    reg_lambda=0,
    reg_alpha=1.0,
    n_estimators=700,
    min_child_weight=3,
    max_depth=10,
    learning_rate=0.1,
    gamma=0.4,
    colsample_bytree=0.7,
    random_state=42
)

print("Entrenando modelo XGBoost...")
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calcular métricas
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

metrics = {
    'ROC_AUC': roc_auc_score(y_test, y_pred_proba),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'Specificity': specificity,
    'F1_Score': f1_score(y_test, y_pred),
    'Accuracy': accuracy_score(y_test, y_pred)
}

# Calcular datos para curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Crear directorio para resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_dir = f'D:/Tesis/Definitivos/nuevo/xgboost_results_{timestamp}'
os.makedirs(result_dir, exist_ok=True)

# 1. Graficar y guardar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - XGBoost')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.savefig(f'{result_dir}/confusion_matrix.png')
plt.close()

# 2. Graficar y guardar curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - XGBoost')
plt.legend(loc="lower right")
plt.savefig(f'{result_dir}/roc_curve.png')
plt.close()

# 3. Graficar Recall y Especificidad
plt.figure(figsize=(8, 6))
metrics_plot = [metrics['Recall'], metrics['Specificity']]
plt.bar(['Recall (Sensibilidad)', 'Especificidad'], metrics_plot, color=['skyblue', 'lightgreen'])
plt.title('Recall vs Especificidad - XGBoost')
plt.ylim([0, 1])
for i, v in enumerate(metrics_plot):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.savefig(f'{result_dir}/recall_specificity.png')
plt.close()

# Guardar todos los resultados
results = {
    'model_name': 'XGBoost',
    'hyperparameters': {
        'subsample': 0.9,
        'reg_lambda': 0.1,
        'reg_alpha': 1.0,
        'n_estimators': 600,
        'min_child_weight': 5,
        'max_depth': 10,
        'learning_rate': 0.1,
        'gamma': 0,
        'colsample_bytree': 0.6
    },
    'metrics': metrics,
    'confusion_matrix': cm.tolist(),
    'class_distribution': {
        'train': {
            'class_0': int(sum(y_train == 0)),
            'class_1': int(sum(y_train == 1))
        }
    },
    'roc_data': {
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist(),
        'roc_auc': roc_auc,
        'thresholds': thresholds.tolist()
    },
    'sensitivity_specificity': {
        'recall': float(metrics['Recall']),
        'specificity': float(metrics['Specificity']),
        'confusion_matrix_values': {
            'tn': int(tn),
            'fp': int(fp),
            'fn': int(fn),
            'tp': int(tp)
        }
    }
}

# Guardar resultados en JSON
with open(f'{result_dir}/model_results.json', 'w') as f:
    json.dump(results, f, indent=4)

# Imprimir resultados en consola
print("\nResultados del modelo XGBoost:")
print("="*40)
print("\nMétricas principales:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nMatriz de Confusión:")
print(cm)

print(f"\nResultados guardados en: {result_dir}")

Entrenando modelo XGBoost...

Resultados del modelo XGBoost:

Métricas principales:
ROC_AUC: 0.9998
Precision: 0.9911
Recall: 0.9770
Specificity: 0.9995
F1_Score: 0.9840
Accuracy: 0.9982

Matriz de Confusión:
[[459489    246]
 [   648  27497]]

Resultados guardados en: D:/Tesis/Definitivos/nuevo/xgboost_results_20250204_020538


# GRADIENT BOOSTING CLASSIFIER

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (roc_curve, auc, roc_auc_score, precision_score, 
                           recall_score, f1_score, accuracy_score, confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import json

# Cargar datos
df_mora_prestamos = pd.read_csv('D:/Tesis/Definitivos/nuevo/df_mora_prestamospersonales.csv')
X = df_mora_prestamos.drop('Morosidad', axis=1)
y = df_mora_prestamos['Morosidad']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Configurar y entrenar modelo con los mejores hiperparámetros
model = GradientBoostingClassifier(
    subsample=1.0,
    n_estimators=700,
    min_samples_split=5,
    min_samples_leaf=4,
    max_features='sqrt',
    max_depth=7,
    learning_rate=0.05,
    random_state=42
)

print("Entrenando modelo Gradient Boosting...")
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calcular métricas
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

metrics = {
    'ROC_AUC': roc_auc_score(y_test, y_pred_proba),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'Specificity': specificity,
    'F1_Score': f1_score(y_test, y_pred),
    'Accuracy': accuracy_score(y_test, y_pred)
}

# Calcular datos para curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Crear directorio para resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_dir = f'D:/Tesis/Definitivos/nuevo/gradient_boosting_results_{timestamp}'
os.makedirs(result_dir, exist_ok=True)

# 1. Graficar y guardar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - Gradient Boosting')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.savefig(f'{result_dir}/confusion_matrix.png')
plt.close()

# 2. Graficar y guardar curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - Gradient Boosting')
plt.legend(loc="lower right")
plt.savefig(f'{result_dir}/roc_curve.png')
plt.close()

# 3. Graficar Recall y Especificidad
plt.figure(figsize=(8, 6))
metrics_plot = [metrics['Recall'], metrics['Specificity']]
plt.bar(['Recall (Sensibilidad)', 'Especificidad'], metrics_plot, color=['skyblue', 'lightgreen'])
plt.title('Recall vs Especificidad - Gradient Boosting')
plt.ylim([0, 1])
for i, v in enumerate(metrics_plot):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.savefig(f'{result_dir}/recall_specificity.png')
plt.close()

# Guardar todos los resultados
results = {
    'model_name': 'Gradient_Boosting',
    'hyperparameters': {
        'subsample': 0.9,
        'n_estimators': 500,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
        'max_features': 'log2',
        'max_depth': 7,
        'learning_rate': 0.05
    },
    'metrics': metrics,
    'confusion_matrix': cm.tolist(),
    'class_distribution': {
        'train': {
            'class_0': int(sum(y_train == 0)),
            'class_1': int(sum(y_train == 1))
        }
    },
    'roc_data': {
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist(),
        'roc_auc': roc_auc,
        'thresholds': thresholds.tolist()
    },
    'sensitivity_specificity': {
        'recall': float(metrics['Recall']),
        'specificity': float(metrics['Specificity']),
        'confusion_matrix_values': {
            'tn': int(tn),
            'fp': int(fp),
            'fn': int(fn),
            'tp': int(tp)
        }
    }
}

# Guardar resultados en JSON
with open(f'{result_dir}/model_results.json', 'w') as f:
    json.dump(results, f, indent=4)

# Imprimir resultados en consola
print("\nResultados del modelo Gradient Boosting:")
print("="*40)
print("\nMétricas principales:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nMatriz de Confusión:")
print(cm)

print(f"\nResultados guardados en: {result_dir}")

Entrenando modelo Gradient Boosting...

Resultados del modelo Gradient Boosting:

Métricas principales:
ROC_AUC: 0.9995
Precision: 0.9867
Recall: 0.9727
Specificity: 0.9992
F1_Score: 0.9797
Accuracy: 0.9977

Matriz de Confusión:
[[459367    368]
 [   768  27377]]

Resultados guardados en: D:/Tesis/Definitivos/nuevo/gradient_boosting_results_20250204_025001


# LIGHTGBM

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import (roc_curve, auc, roc_auc_score, precision_score, 
                           recall_score, f1_score, accuracy_score, confusion_matrix)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import json

# Cargar datos
df_mora_prestamos = pd.read_csv('D:/Tesis/Definitivos/nuevo/df_mora_prestamospersonales.csv')
X = df_mora_prestamos.drop('Morosidad', axis=1)
y = df_mora_prestamos['Morosidad']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Configurar y entrenar modelo con los mejores hiperparámetros
model = lgb.LGBMClassifier(
    subsample=0.6,
    reg_lambda=0.1,
    reg_alpha=0.1,
    num_leaves=127,
    n_estimators=900,
    min_child_samples=10,
    max_depth=-1,
    learning_rate=0.05,
    colsample_bytree=0.6,
    random_state=42
)

print("Entrenando modelo LightGBM...")
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calcular métricas
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()
specificity = tn / (tn + fp)

metrics = {
    'ROC_AUC': roc_auc_score(y_test, y_pred_proba),
    'Precision': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'Specificity': specificity,
    'F1_Score': f1_score(y_test, y_pred),
    'Accuracy': accuracy_score(y_test, y_pred)
}

# Calcular datos para curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Crear directorio para resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
result_dir = f'D:/Tesis/Definitivos/nuevo/lightgbm_results_{timestamp}'
os.makedirs(result_dir, exist_ok=True)

# 1. Graficar y guardar matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión - LightGBM')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.savefig(f'{result_dir}/confusion_matrix.png')
plt.close()

# 2. Graficar y guardar curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC - LightGBM')
plt.legend(loc="lower right")
plt.savefig(f'{result_dir}/roc_curve.png')
plt.close()

# 3. Graficar Recall y Especificidad
plt.figure(figsize=(8, 6))
metrics_plot = [metrics['Recall'], metrics['Specificity']]
plt.bar(['Recall (Sensibilidad)', 'Especificidad'], metrics_plot, color=['skyblue', 'lightgreen'])
plt.title('Recall vs Especificidad - LightGBM')
plt.ylim([0, 1])
for i, v in enumerate(metrics_plot):
    plt.text(i, v + 0.01, f'{v:.4f}', ha='center')
plt.savefig(f'{result_dir}/recall_specificity.png')
plt.close()

# Guardar todos los resultados
results = {
    'model_name': 'LightGBM',
    'hyperparameters': {
        'subsample': 0.6,
        'reg_lambda': 0.1,
        'reg_alpha': 0.1,
        'num_leaves': 127,
        'n_estimators': 900,
        'min_child_samples': 10,
        'max_depth': -1,
        'learning_rate': 0.05,
        'colsample_bytree': 0.6
    },
    'metrics': metrics,
    'confusion_matrix': cm.tolist(),
    'class_distribution': {
        'train': {
            'class_0': int(sum(y_train == 0)),
            'class_1': int(sum(y_train == 1))
        }
    },
    'roc_data': {
        'fpr': fpr.tolist(),
        'tpr': tpr.tolist(),
        'roc_auc': roc_auc,
        'thresholds': thresholds.tolist()
    },
    'sensitivity_specificity': {
        'recall': float(metrics['Recall']),
        'specificity': float(metrics['Specificity']),
        'confusion_matrix_values': {
            'tn': int(tn),
            'fp': int(fp),
            'fn': int(fn),
            'tp': int(tp)
        }
    }
}

# Guardar resultados en JSON
with open(f'{result_dir}/model_results.json', 'w') as f:
    json.dump(results, f, indent=4)

# Imprimir resultados en consola
print("\nResultados del modelo LightGBM:")
print("="*40)
print("\nMétricas principales:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

print("\nMatriz de Confusión:")
print(cm)

print(f"\nResultados guardados en: {result_dir}")

Entrenando modelo LightGBM...
[LightGBM] [Info] Number of positive: 65670, number of negative: 1072714
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.096734 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3625
[LightGBM] [Info] Number of data points in the train set: 1138384, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.057687 -> initscore=-2.793305
[LightGBM] [Info] Start training from score -2.793305

Resultados del modelo LightGBM:

Métricas principales:
ROC_AUC: 0.9998
Precision: 0.9921
Recall: 0.9787
Specificity: 0.9995
F1_Score: 0.9853
Accuracy: 0.9983

Matriz de Confusión:
[[459515    220]
 [   600  27545]]

Resultados guardados en: D:/Tesis/Definitivos/nuevo/lightgbm_results_20250204_025221


# COMPARACIÓN DE RESULTADOS

In [43]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from glob import glob

def find_latest_results(base_dir, model_prefix):
    """Encuentra el directorio de resultados más reciente para cada modelo"""
    pattern = os.path.join(base_dir, f"{model_prefix}_results_*")
    dirs = glob(pattern)
    if dirs:
        return max(dirs, key=os.path.getctime)
    return None

def load_model_results(base_dir):
    """Cargar resultados de todos los modelos"""
    all_results = {}
    
    # Lista de prefijos de modelos
    model_prefixes = [
        'logistic_regression',
        'random_forest',
        'xgboost',
        'lightgbm',
        'neural_network',
        'gradient_boosting'
    ]
    
    for prefix in model_prefixes:
        latest_dir = find_latest_results(base_dir, prefix)
        if latest_dir:
            try:
                results_file = os.path.join(latest_dir, 'model_results.json')
                if os.path.exists(results_file):
                    with open(results_file, 'r') as f:
                        results = json.load(f)
                        model_name = results['model_name']
                        all_results[model_name] = results
                        print(f"Cargados resultados de {model_name}")
            except Exception as e:
                print(f"Error cargando resultados de {prefix}: {str(e)}")
    
    if not all_results:
        raise ValueError("No se encontraron resultados de modelos")
    
    return all_results

def plot_roc_comparison(all_results):
    """Graficar comparación de curvas ROC"""
    plt.figure(figsize=(12, 8))
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEEAD', '#D4A5A5']
    
    for (model_name, results), color in zip(all_results.items(), colors):
        fpr = np.array(results['roc_data']['fpr'])
        tpr = np.array(results['roc_data']['tpr'])
        roc_auc = results['roc_data']['roc_auc']
        
        plt.plot(fpr, tpr, color=color, lw=2,
                label=f'{model_name} (AUC = {roc_auc:.4f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.xlabel('Tasa de Falsos Positivos', fontsize=12)
    plt.ylabel('Tasa de Verdaderos Positivos', fontsize=12)
    plt.title('Comparación de Curvas ROC', fontsize=14)
    plt.legend(loc='lower right', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.savefig('comparison_roc_curves.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_metrics_comparison(all_results):
    """Graficar comparación de métricas principales"""
    # Crear DataFrame con las métricas
    metrics_data = {}
    for model_name, results in all_results.items():
        metrics_data[model_name] = results['metrics']
    
    metrics_df = pd.DataFrame(metrics_data).T
    
    # Crear heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(metrics_df, annot=True, fmt='.4f', cmap='YlOrRd', center=0.5)
    plt.title('Comparación de Métricas', fontsize=14)
    plt.tight_layout()
    plt.savefig('comparison_metrics_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_recall_specificity_comparison(all_results):
    """Graficar comparación de Recall y Especificidad"""
    models = list(all_results.keys())
    recalls = [results['metrics']['Recall'] for results in all_results.values()]
    specificities = [results['metrics']['Specificity'] for results in all_results.values()]

    x = np.arange(len(models))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 6))
    rects1 = ax.bar(x - width/2, recalls, width, label='Recall', color='skyblue')
    rects2 = ax.bar(x + width/2, specificities, width, label='Especificidad', color='lightgreen')

    ax.set_xlabel('Modelos', fontsize=12)
    ax.set_ylabel('Valor', fontsize=12)
    ax.set_title('Comparación de Recall y Especificidad', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(models, rotation=45, ha='right')
    ax.legend()

    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate(f'{height:.4f}',
                       xy=(rect.get_x() + rect.get_width()/2, height),
                       xytext=(0, 3),
                       textcoords="offset points",
                       ha='center', va='bottom')

    autolabel(rects1)
    autolabel(rects2)

    fig.tight_layout()
    plt.savefig('comparison_recall_specificity.png', dpi=300, bbox_inches='tight')
    plt.close()

def plot_confusion_matrices(all_results):
    """Graficar todas las matrices de confusión"""
    n_models = len(all_results)
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()

    for idx, (model_name, results) in enumerate(all_results.items()):
        cm = np.array(results['confusion_matrix'])
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
        axes[idx].set_title(f'Matriz de Confusión - {model_name}')
        axes[idx].set_ylabel('Real')
        axes[idx].set_xlabel('Predicho')

    plt.tight_layout()
    plt.savefig('comparison_confusion_matrices.png', dpi=300, bbox_inches='tight')
    plt.close()

def main():
    base_dir = 'D:/Tesis/Definitivos/nuevo'
    print("Cargando resultados de los modelos...")
    
    try:
        all_results = load_model_results(base_dir)
        
        print(f"\nModelos encontrados: {list(all_results.keys())}")
        
        print("\nGenerando visualizaciones comparativas...")
        plot_roc_comparison(all_results)
        plot_metrics_comparison(all_results)
        plot_recall_specificity_comparison(all_results)
        plot_confusion_matrices(all_results)
        
        # Crear tabla resumen
        metrics_df = pd.DataFrame({name: res['metrics'] 
                                 for name, res in all_results.items()}).round(4)
        
        print("\nResumen de métricas:")
        print(metrics_df)
        metrics_df.to_csv('metrics_summary.csv')
        
        print("\nVisualizaciones guardadas:")
        print("- comparison_roc_curves.png")
        print("- comparison_metrics_heatmap.png")
        print("- comparison_recall_specificity.png")
        print("- comparison_confusion_matrices.png")
        print("- metrics_summary.csv")
        
    except Exception as e:
        print(f"Error en la ejecución: {str(e)}")

if __name__ == "__main__":
    main()

Cargando resultados de los modelos...
Cargados resultados de Logistic_Regression
Cargados resultados de Random_Forest
Cargados resultados de XGBoost
Cargados resultados de LightGBM
Cargados resultados de Neural_Network
Cargados resultados de Gradient_Boosting

Modelos encontrados: ['Logistic_Regression', 'Random_Forest', 'XGBoost', 'LightGBM', 'Neural_Network', 'Gradient_Boosting']

Generando visualizaciones comparativas...

Resumen de métricas:
             Logistic_Regression  Random_Forest  XGBoost  LightGBM  \
ROC_AUC                   0.9883         0.9996   0.9998    0.9998   
Precision                 0.6193         0.9801   0.9911    0.9921   
Recall                    0.9571         0.9669   0.9770    0.9787   
Specificity               0.9645         0.9988   0.9995    0.9995   
F1_Score                  0.7520         0.9735   0.9840    0.9853   
Accuracy                  0.9641         0.9970   0.9982    0.9983   

             Neural_Network  Gradient_Boosting  
ROC_AUC   